# *Data aggregation and cleaning of fishing datasets*

# Step 1: Cleaning the data

## First we'll try to create tables from the seperate .txt files using the Pandas library

In [1]:
import pandas as pd

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os, sys

pd.options.display.max_rows = 8

In [3]:
#Code provided by Wai Kit to load in all the filenames
files = os.listdir('Data/csv_sheets/')
files = ['Data/csv_sheets/' + file for file in files] 
files

['Data/csv_sheets/201201.csv',
 'Data/csv_sheets/201202.csv',
 'Data/csv_sheets/201203.csv',
 'Data/csv_sheets/201204.csv',
 'Data/csv_sheets/201205.csv',
 'Data/csv_sheets/201206.csv',
 'Data/csv_sheets/201207.csv',
 'Data/csv_sheets/201208.csv',
 'Data/csv_sheets/201209.csv',
 'Data/csv_sheets/201210.csv',
 'Data/csv_sheets/201211.csv',
 'Data/csv_sheets/201212.csv',
 'Data/csv_sheets/201301.csv',
 'Data/csv_sheets/201302.csv',
 'Data/csv_sheets/201303.csv',
 'Data/csv_sheets/201304.csv',
 'Data/csv_sheets/201305.csv',
 'Data/csv_sheets/201306.csv',
 'Data/csv_sheets/201307.csv',
 'Data/csv_sheets/201308.csv',
 'Data/csv_sheets/201309.csv',
 'Data/csv_sheets/201310.csv',
 'Data/csv_sheets/201311.csv',
 'Data/csv_sheets/201312.csv',
 'Data/csv_sheets/201401.csv',
 'Data/csv_sheets/201402.csv',
 'Data/csv_sheets/201403.csv',
 'Data/csv_sheets/201404.csv',
 'Data/csv_sheets/201405.csv',
 'Data/csv_sheets/201406.csv',
 'Data/csv_sheets/201407.csv',
 'Data/csv_sheets/201408.csv',
 'Data/c

## In the following cells we'll be trying the necessary operations for the data cleaning step, on a single datafile (equivalent of 1 month of data)

In [71]:
#Load in the data for January 2016
path = 'Data/csv_sheets/201601.csv'
data_201601 = pd.read_csv(path , skiprows = 8, error_bad_lines=False)
data_201601


b'Skipping line 64: expected 10 fields, saw 11\n'


,8,Totaal,1656417.000000,5292876.000000,544670.000000,1667292.000000,1110233.000000,3617274.000000,1514.000000,8312.000000
0,9,Schelvis,10839.000000,22150.000000,676.000000,2116.000000,10163.000000,20034.000000,-,-
1,10,Kabeljauw,48202.000000,158732.000000,13301.000000,46088.000000,34295.000000,110472.000000,606.000000,2172.000000
2,11,Koolvis,376.000000,606.000000,16.000000,22.000000,360.000000,586.000000,-,-
3,12,Wijting,28187.000000,37956.000000,9756.000000,13908.000000,18381.000000,24012.000000,50.000000,36.000000
...,...,...,...,...,...,...,...,...,...,...
51,60,Wulk,4347.000000,3416.000000,1408.000000,1338.000000,2939.000000,2078.000000,-,-
52,61,Andere soorten,-,-,-,-,-,-,-,-
53,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_201601.columns = ["Index", "Soort", "Totaal in kg", "Totaal in EUR", "Oostende in kg", "Oostende in EUR", "Zeebrugge in kg", "Zeebrugge in EUR", "Nieuwpoort in kg", "Nieuwpoort in EUR"]
data_201601

In [ ]:
data_201601 = data_201601.drop('Index', 1)
data_201601

In [ ]:
data_201601 = data_201601[:-2]
data_201601

In [ ]:
data_201601 = data_201601.drop(data_201601.index[[32,33,34,40,41,42]])
data_201601.head(n=36)

### In order to round all the numbers after the comma, we will first check the format/types of the cells.

In [ ]:
data_201601.dtypes

### We now know that the imported data are all of 'Object' type, and will need to convert them to 'integer' in order to be able to round them. 

In [ ]:
data_201601 = data_201601.convert_objects(convert_numeric=True)
data_201601.dtypes

### Now we'll replace the NaN values in our dataframe with zero's, since zero kg/EUR of a certain fish species where caught in that certain month

In [ ]:
data_201601 = data_201601.fillna(0)
data_201601

### We will create a new variable that gives us the family to which the certain species belongs

In [ ]:
data_201601.loc[0:32, 'Familie']= 'Bodemvis'
data_201601.loc[34:39, 'Familie']= 'Pelagische Vis'
data_201601.loc[42:52, 'Familie'] = 'Schaal- en Weekdieren'
data_201601


### The next step in our data aggregation is to create a seperate row for each observation. I.e. we want to have 3 'Schelvis' observations, one for every harbor (per month).

#### * We can drop the columns that represent the total fish caught over all harbors *

In [ ]:
data_201601 = data_201601.drop(data_201601.columns[[1, 2]], 1)
data_201601

In [ ]:
data_201601_kg = pd.melt(data_201601, id_vars=['Soort','Familie'], value_vars=['Oostende in kg', 'Zeebrugge in kg', 'Nieuwpoort in kg'], var_name='Haven', value_name = 'Hoeveelheid in kg')
data_201601_EUR = pd.melt(data_201601, id_vars=['Soort','Familie'], value_vars=['Oostende in EUR', 'Zeebrugge in EUR', 'Nieuwpoort in EUR'], var_name='Haven', value_name = 'Hoeveelheid in EU')

In [ ]:
data_201601_EUR['Haven'] = data_201601_EUR['Haven'].map(lambda x: str(x)[:-7])
data_201601_EUR

In [ ]:
data_201601_kg['Haven'] = data_201601_kg['Haven'].map(lambda x: str(x)[:-6])
data_201601_kg

In [ ]:
data_201601 = pd.merge(data_201601_kg, data_201601_EUR, how='left', on=['Soort', 'Familie', 'Haven'])

In [ ]:
data_201601.loc[0:50]

In [74]:
# We will substract our date variables from the filename '201601'
path

'Data/csv_sheets/201601.csv'

In [ ]:
# substring for the year variable
path[-10:-6]

In [ ]:
# substring for the month variable
path[-9:-7]

In [75]:
data_201601['year'] = path[-10:-6]
data_201601['month'] = path[-6:-4]

In [76]:
data_201601

,8,Totaal,1656417.000000,5292876.000000,544670.000000,1667292.000000,1110233.000000,3617274.000000,1514.000000,8312.000000,year,month
0,9,Schelvis,10839.000000,22150.000000,676.000000,2116.000000,10163.000000,20034.000000,-,-,2016,01
1,10,Kabeljauw,48202.000000,158732.000000,13301.000000,46088.000000,34295.000000,110472.000000,606.000000,2172.000000,2016,01
2,11,Koolvis,376.000000,606.000000,16.000000,22.000000,360.000000,586.000000,-,-,2016,01
3,12,Wijting,28187.000000,37956.000000,9756.000000,13908.000000,18381.000000,24012.000000,50.000000,36.000000,2016,01
...,...,...,...,...,...,...,...,...,...,...,...,...
51,60,Wulk,4347.000000,3416.000000,1408.000000,1338.000000,2939.000000,2078.000000,-,-,2016,01
52,61,Andere soorten,-,-,-,-,-,-,-,-,2016,01
53,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,01
54,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,01


#### *We now have 1 observation per row, for the whole month January 2016*

# Step 2: Data Aggregation

## Now we will have to do these operations on all the stored csv files, and add them together to get our final table

In [4]:
files[1]

'Data/csv_sheets/201202.csv'

In [ ]:
#how to add each new table to the existing final table
# for file in files:
#    data = pd.read_cs# v(file, skiprows = 8, error_bad_lines=False)
#    final_table = final_table.append(data)

### In the next cell we'll create a for loop where we'll read in the files and execute all the above data-cleaning operations. At the end of each loop, we will append the current table to all the previous handled tables.

In [103]:
final_table = pd.DataFrame()

In [104]:
for file in files:
    data = pd.read_csv(file, skiprows = 8, error_bad_lines=False)
    
    data.columns = ["Index", "Soort", "Totaal in kg", "Totaal in EUR", "Oostende in kg", "Oostende in EUR", "Zeebrugge in kg", "Zeebrugge in EUR", "Nieuwpoort in kg", "Nieuwpoort in EUR"]
    data = data.drop('Index', 1)
    data = data[:-2]
    data = data.drop(data.index[[32,33,34,40,41,42]])
    data = data.convert_objects(convert_numeric=True)
    data = data.fillna(0)
    data.loc[0:32, 'Familie']= 'Bodemvis'
    data.loc[34:39, 'Familie']= 'Pelagische Vis'
    data.loc[42:52, 'Familie'] = 'Schaal- en Weekdieren'
    data = data.drop(data.columns[[1, 2]], 1)
    data_kg = pd.melt(data, id_vars=['Soort','Familie'], value_vars=['Oostende in kg', 'Zeebrugge in kg', 'Nieuwpoort in kg'], var_name='Haven', value_name = 'Hoeveelheid in kg')
    data_EUR = pd.melt(data, id_vars=['Soort','Familie'], value_vars=['Oostende in EUR', 'Zeebrugge in EUR', 'Nieuwpoort in EUR'], var_name='Haven', value_name = 'Hoeveelheid in EU')
    data_EUR['Haven'] = data_EUR['Haven'].map(lambda x: str(x)[:-7])
    data_kg['Haven'] = data_kg['Haven'].map(lambda x: str(x)[:-6])
    data = pd.merge(data_kg, data_EUR, how='left', on=['Soort', 'Familie', 'Haven'])
    data = pd.merge(data_kg, data_EUR, how='left', on=['Soort', 'Familie', 'Haven'])
    data['year'] = file[-10:-6]
    data['month'] = file[-6:-4]

    final_table = final_table.append(data)
    
final_table

C:\Users\Frederik Stallaert\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
b'Skipping line 64: expected 10 fields, saw 11\n'
b'Skipping line 64: expected 10 fields, saw 11\n'
b'Skipping line 64: expected 10 fields, saw 11\n'
b'Skipping line 64: expected 10 fields, saw 11\n'
b'Skipping line 64: expected 10 fields, saw 11\n'
b'Skipping line 64: expected 10 fields, saw 11\n'
b'Skipping line 64: expected 10 fields, saw 11\n'


,Soort,Familie,Haven,Hoeveelheid in kg,Hoeveelheid in EU,year,month
0,Schelvis,Bodemvis,Oostende,4782.0,5250.0,2012,01
1,Kabeljauw,Bodemvis,Oostende,22449.0,58882.0,2012,01
2,Koolvis,Bodemvis,Oostende,6.0,16.0,2012,01
3,Wijting,Bodemvis,Oostende,5495.0,6246.0,2012,01
...,...,...,...,...,...,...,...
137,Schelpen,Schaal- en Weekdieren,Nieuwpoort,0.0,0.0,2016,07
138,Inktvis,Schaal- en Weekdieren,Nieuwpoort,19.0,60.0,2016,07
139,Wulk,Schaal- en Weekdieren,Nieuwpoort,0.0,0.0,2016,07
140,Andere soorten,Schaal- en Weekdieren,Nieuwpoort,0.0,0.0,2016,07


In [108]:
final_table.iloc[40:48]

,Soort,Familie,Haven,Hoeveelheid in kg,Hoeveelheid in EU,year,month
42,Schelpen,Schaal- en Weekdieren,Oostende,26008.0,58408.0,2012,01
43,Inktvis,Schaal- en Weekdieren,Oostende,30921.0,91556.0,2012,01
44,Wulk,Schaal- en Weekdieren,Oostende,1573.0,1040.0,2012,01
45,Andere soorten,Schaal- en Weekdieren,Oostende,0.0,0.0,2012,01
47,Schelvis,Bodemvis,Zeebrugge,24088.0,27700.0,2012,01
48,Kabeljauw,Bodemvis,Zeebrugge,59347.0,142262.0,2012,01
49,Koolvis,Bodemvis,Zeebrugge,30.0,60.0,2012,01
50,Wijting,Bodemvis,Zeebrugge,22134.0,26972.0,2012,01


In [105]:
final_table=final_table[final_table.Soort != 0]

In [119]:
final_table.to_csv('cleaned_data.csv', index=False)